# ReadMe

The code below was written for the UNSW Data Science Society Blog by Julian Garratt.

Blog Post: Trading Stocks Based on Sentiment

Note: No need to change any code unless stated otherwise (e.g. to change certain parameters)

In [ ]:
# Downloaded any other libraries here
!pip install newsapi-python
!pip install openpyxl
!pip install yahoo_fin
!pip install nltk
!pip install xlrd

In [ ]:
# Libraries
import numpy as np 
import pandas as pd 
import requests
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
# IMPORTANT: Get API Key (free) at https://newsapi.org/ 
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key='INSERT API KEY')


In [ ]:
"""
from_param = choose starting date to scrap news
to = choose ending date
Other parameters don't need to be changed.
"""
every_headline = newsapi.get_everything(q='asx',
                                         sources="abc-news-au",
                                         from_param = "2021-03-20",
                                         to = "2021-03-22",
                                         sort_by="relevancy",
                                         language='en')
print("Done: {} results found".format(every_headline["totalResults"]))

In [ ]:
# Map ASX companies to a list of tuples [(company, code),...]
asx_mapping = pd.read_excel("asx_code_to_company.xlsx", engine='openpyxl')
asx_mapping.Company = asx_mapping.Company.str.replace(" ltd", "", case = False)
asx_companies = list(zip(asx_mapping.Company.str.lower().values,asx_mapping.Code.values))
asx_companies[:10]

In [ ]:
"""
n_urls can be changed to be greater than or less that 3 (as long as there exists more than 3 scraped news articles)
"""

# Extract only content from "n" urls from the news api call (i.e. get rid of unimportant stuff e.g. javascript)
def get_content(n_urls):
    content = []
    
    for n in range(n_urls):
        # Get html of most relevant news article
        r = requests.get(every_headline["articles"][n]["url"])
        text = r.text
        matches = [(m.start(0), m.end(0)) for m in re.finditer("{\"type\":\"text\",\"content\":\"(.*?)\"}", text)]

        for start, end in matches:
            content.append(eval(text[start:end])["content"])
    
    return content

content = get_content(3)

In [ ]:
# Create dict of {Company : news about company, ...}
company_news_dict = {}
for company, code in asx_companies:
    r_company = re.compile(fr"\b({str(company)})\b")
    r_code = re.compile(fr"\b({str(code)})\b")
    company_news_dict[str(code)] = [line for line in content if r_company.search(line.lower()) or r_code.search(line)]

# Remove values where a company has no news
keys_to_remove = []
for key in company_news_dict.keys():
    if not company_news_dict[key]:
        keys_to_remove.append(key)

for keys in keys_to_remove:
    company_news_dict.pop(keys)

company_news_dict

In [ ]:
# Calculate sentiment score for each company and create dict of {code : mean sentiment...}
sid = SentimentIntensityAnalyzer()

def polarity_score(sentence, sid):
    sentiment_dict = sid.polarity_scores(sentence)
    return sentiment_dict["compound"]

asx_sentiment_dict = {}
for code in company_news_dict.keys():
    sentiment = []
    for sentence in company_news_dict[code]:
        sentiment.append(polarity_score(sentence, sid))

    mean_sentiment = sum(sentiment)/len(sentiment)
    asx_sentiment_dict[code] = mean_sentiment

# remove ABC (ABC in this context does not stand for Australian Broadcasting Corporation) as it occurs frequently in ABC articles
asx_sentiment_dict.pop("ABC", None) 
asx_sentiment_dict

# Backtesting


In [ ]:
from yahoo_fin.stock_info import get_data

In [ ]:
class backtest():
    
    def __init__(self, asx_sentiment_dict, capital, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self.codes = [key for key in asx_sentiment_dict.keys() if asx_sentiment_dict[key] > 0]
        self.capital_per_share = capital / len(self.codes)
        self.capital = capital
        self.ticker_data_dict = self.get_ticker_data(asx_sentiment_dict)
        self.delta_time = len(list(self.ticker_data_dict.items())[0][1].index)
        self.init_value = self.get_init_value()
        
    def get_ticker_data(self, asx_sentiment_dict):
        ticker_data_dict = {}
        for code in asx_sentiment_dict.keys():
            if asx_sentiment_dict[code] > 0:
                ticker_data_dict[code] = get_data("{}.AX".format(code), start_date = self.start_date, end_date = self.end_date)
        return ticker_data_dict
    
    def get_init_value(self):
        init_value = {}
        for code in self.codes:
            init_value[code] = self.capital_per_share / self.ticker_data_dict[code].open.iloc[0]
        return init_value
    
    def backtest(self):
        # Find value per day per share at open price
        for code in self.codes:
            self.ticker_data_dict[code]["value"] = self.ticker_data_dict[code].open * self.init_value[code]
        # Find total value of portfolio per day
        total_value_per_day = []
        for i in range(self.delta_time):
            value_per_share = []
            for code in self.codes:
                code_value = self.ticker_data_dict[code]["value"].iloc[i]
                value_per_share.append(code_value)
            total_value_per_day.append(sum(value_per_share))
        # Plot results
        print("Final Value = {:.2f}".format(total_value_per_day[-1]))
        print("Gains/Losses = {:.2f}".format(total_value_per_day[-1] - total_value_per_day[0]))
        sns.set(style='whitegrid', context='talk')
        sns.lineplot(x = list(self.ticker_data_dict.items())[0][1].index.to_list(), y = total_value_per_day)
        sns.despine()
        plt.axhline(y=self.capital, label = "Baseline Capital", color = "red", linestyle = "--")
        plt.xlabel("Date")
        plt.ylabel("Value")
        actual_start_date = pd.to_datetime(self.start_date, format="%m/%d/%Y") - pd.DateOffset(1)
        actual_end_date = pd.to_datetime(self.end_date, format="%m/%d/%Y") - pd.DateOffset(1)
        plt.title("Portfolio Value from {} to {}".format(actual_start_date.strftime("%d/%m/%Y"), actual_end_date.strftime("%d/%m/%Y")))
        plt.xticks(rotation = 45)
        plt.show()

In [ ]:
"""
capital = Initial value of the portfolio
start_date = Date to begin simulation
end_date = Date to end simulation
"""

# Backtest strategy
# Dates structure: %m/%d/%Y (will always get data from: start_date & end_date minus 1)
backtester = backtest(asx_sentiment_dict, capital=100000, start_date="03/24/2021", end_date="03/31/2021")

In [ ]:
# Executing this block of code will run the backtest and will plot the portfolio value over time and final metrics.
backtester.backtest()